In [1]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

## Loading Training/Testing/Validation Data

In [2]:
dataset = 'PMNIST'

x_train_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'x_test.npz')))
x_valid_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'x_valid.npz')))

x_train = x_train_load['arr_0']
x_test = x_test_load['arr_0']
x_valid = x_valid_load['arr_0']

y_train_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'y_test.npz')))
y_valid_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'y_valid.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']
y_valid = y_valid_load['arr_0']

print(f"x_train: {x_train.shape} - y_train: {y_train.shape}")
print(f"x_test: {x_test.shape} - y_test: {y_test.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_train: (50000, 784, 1) - y_train: (50000,)
x_test: (10000, 784, 1) - y_test: (10000,)
x_valid: (10000, 784, 1) - y_valid: (10000,)


In [3]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

model = Sequential()

model.add(RNN(NeuromodulatedBistableRecurrentCellLayer(output_dim = 212),
              input_shape = (None, 1), 
              return_sequences = False))
model.add(Dense(to_categorical(y_train).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 212)               90948     
_________________________________________________________________
dense (Dense)                (None, 10)                2130      
Total params: 93,078
Trainable params: 93,078
Non-trainable params: 0
_________________________________________________________________


In [4]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-nBRC.hdf5'))

callbacks = [
    ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
    EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min')]

t = time.time()

result = model.fit(x_train, 
                   to_categorical(y_train),
                   epochs = 200, 
                   batch_size = 100, 
                   validation_data = (x_valid, to_categorical(y_valid)), 
                   callbacks = callbacks)

print(f"Training time: {time.time() - t} s")

df_results = pd.DataFrame(result.history)
df_results.to_csv(os.path.abspath(os.path.join('models', dataset, 'nBRC_results.csv')))

Train on 50000 samples, validate on 10000 samples
Epoch 1/200
49900/50000 [============================>.] - ETA: 0s - loss: 1.9356 - acc: 0.3098
Epoch 00001: val_loss improved from inf to 1.56647, saving model to /data/home/dorads/Documents/GitHub_Repos/01_Maintained/NeuralODE/models/PMNIST/PMNIST-nBRC.hdf5
50000/50000 [==============================] - 306s 6ms/sample - loss: 1.9351 - acc: 0.3101 - val_loss: 1.5665 - val_acc: 0.4489
Epoch 2/200
49900/50000 [============================>.] - ETA: 0s - loss: 1.3838 - acc: 0.5292
Epoch 00002: val_loss improved from 1.56647 to 1.20934, saving model to /data/home/dorads/Documents/GitHub_Repos/01_Maintained/NeuralODE/models/PMNIST/PMNIST-nBRC.hdf5
50000/50000 [==============================] - 307s 6ms/sample - loss: 1.3833 - acc: 0.5294 - val_loss: 1.2093 - val_acc: 0.5902
Epoch 3/200
49900/50000 [============================>.] - ETA: 0s - loss: 1.1140 - acc: 0.6260
Epoch 00003: val_loss improved from 1.20934 to 0.98629, saving model to 

In [ ]:
model.load_weights(file_path)
model.evaluate(x_test, to_categorical(y_test))

 9408/10000 [===========================>..] - ETA: 3s - loss: 0.4172 - acc: 0.8690